# Where is the fractal?  v2
- Lorenz attractor in Julia
- no average

In [1]:
using BenchmarkTools
#using Plots

In [2]:
using DifferentialEquations

┌ Info: Recompiling stale cache file /home/etiennenavarro/.julia/compiled/v1.0/DifferentialEquations/UQdwS.ji for DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1190


In [6]:
# Parameters
sigma = 10.0
rho = 28.0
beta = 8/3.;

# Fixed points
p1 = [-sqrt(beta*(rho-1)), -sqrt(beta*(rho-1)), rho-1];
p2 = [+sqrt(beta*(rho-1)), +sqrt(beta*(rho-1)), rho-1];

p12 = p2 .- p1
p12_norm = sqrt(sum(p12.^2))
p12 = p12./p12_norm;

function positive_side(u)
    a = u .- p2
    return  sum(a .* p12) > 0
end
function negative_side(u)
    a = u .- p1
    return  -sum(a .* p12) > 0
end

negative_side (generic function with 1 method)

In [8]:
# test 
println( positive_side([50., 50.0, 50.0]) )
println( positive_side([-50., -50.0, 0.0]) )
println( positive_side([0., 0.0, 0.0]) )
println( negative_side([50., 50.0, 50.0]) )
println( negative_side([-50., -50.0, 0.0]) )
println( negative_side([0., 0.0, 0.0]) )

true
false
false
false
true
false


In [9]:
function lorenz!(du, u, p, t)
    du[1] = sigma*(u[2]-u[1])
    du[2] = u[1]*(rho-u[3]) - u[2]
    du[3] = u[1]*u[2] - beta*u[3]
end

lorenz! (generic function with 1 method)

In [10]:
function lorenz_colored!(du, u, p, t)
    du[1] = sigma*(u[2]-u[1])
    du[2] = u[1]*(rho-u[3]) - u[2]
    du[3] = u[1]*u[2] - beta*u[3]
    du[1:3] = du[1:3] ./ sqrt(sum(du[1:3].^2))  # norm
    
    if positive_side(u[1:3])
        kA = .1
        kB = .0
    elseif negative_side(u[1:3])
        kA = .0
        kB = .1
    else
        kA = .0
        kB = .0
    end
    du[4] = -u[4]*kA + kB*(1-u[4])
end

lorenz_colored! (generic function with 1 method)

In [67]:
# Cube size
xy_half_size = 25
z_size = 45
voxel_size = z_size/2000.0;
dt = .5*voxel_size

0.01125

In [68]:
# Convert x coordinate to one integer
to_index(x, left, right, n) = trunc(Int16, (x - left)/(right - left)*n)

# test
to_index(12, -10, 10, 100)

110

In [69]:
# Same for the 3 coordinates
n_xy = trunc(Int16, 2*xy_half_size/voxel_size)
n_z = trunc(Int16, z_size/voxel_size)

function to_ijk(xyz)
    x, y, z = xyz   
    i = to_index(x, -xy_half_size, +xy_half_size, n_xy)
    j = to_index(y, -xy_half_size, +xy_half_size, n_xy)
    k = to_index(z, 0, +z_size, n_z)
   return  i, j, k
end

# test
to_ijk(rand(3).*10)

(1329, 1128, 275)

In [51]:
@btime to_ijk(rand(3).*10)
#   760.805 ns (12 allocations: 332 bytes)
#   670.129 ns (10 allocations: 312 bytes)
#   500.304 ns (6 allocations: 272 bytes)
#   496.554 ns (6 allocations: 272 bytes)
#   507.297 ns (7 allocations: 280 bytes) --> Int16

  507.297 ns (7 allocations: 280 bytes)


(447, 608, 69)

In [70]:
# To the attractor
function burn()
    u0 = (rand(Float64, 3) .- [0.5, 0.5, 0]).*[2*xy_half_size, 2*xy_half_size, z_size]
    tspan = (0.0, 100.0)
    prob = ODEProblem(lorenz!, u0, tspan)
    tol = 1e-3
    sol = solve(prob, Tsit5(), reltol=tol, abstol=tol,
                save_everystep=false);
    return sol[end]
end

burn (generic function with 1 method)

In [15]:
@btime burn()
#   2.088 ms (81153 allocations: 964.95 KiB)   dell

  2.088 ms (81153 allocations: 964.95 KiB)


3-element Array{Float64,1}:
 -3.605500565109067
 -6.030688026365427
 13.632100463363805

In [71]:
# test
u0 = [burn()..., .5]

tspan = (0.0, 100.0)
prob = ODEProblem(lorenz_colored!, u0, tspan)

tol = 1e-6
sol = solve(prob, Tsit5(), #callback=cb,
            reltol=tol, abstol=tol, saveat=dt);

In [72]:
# Run the solver and fill the cube
function populate!(cube)

    u0 = [burn()..., .5]

    tspan = (0.0, 200.0)
    prob = ODEProblem(lorenz_colored!, u0, tspan)

    tol = 1e-7
    sol = solve(prob, Tsit5(), #callback=cb,
                reltol=tol, abstol=tol, saveat=dt);

    #saved_event_idx
    #println("sol size: ", size(sol.u))

    for xyzc in sol.u
        ijk = to_ijk(xyzc[1:3])
        cube[ijk] = get(cube, ijk, (0, 0.0)) .+ (1, xyzc[4])
    end
    
end

populate! (generic function with 1 method)

In [73]:
@btime populate!(cube)

  37.724 ms (442872 allocations: 11.31 MiB)


In [74]:
# Init the cube
cube = Dict{Tuple{Int16, Int16, Int16}, Tuple{Int64, Float64}}()

Dict{Tuple{Int16,Int16,Int16},Tuple{Int64,Float64}} with 0 entries

In [82]:
for k in 1:5000
    populate!(cube)
    print("\rcube size:  ", length(cube), "\t k: ", k)
end
println("\n min, max= ", extrema([ x[1] for x in values(cube)]))
println("c min, max= ", extrema([ x[2] for x in values(cube)]))

cube size:  8205687	 k: 5000
 min, max= (1, 192)
c min, max= (2.919456563925344e-6, 179.3257266923889)


In [83]:
varinfo(r"cube")

| name |        size | summary                                                                  |
|:---- | -----------:|:------------------------------------------------------------------------ |
| cube | 368.000 MiB | Dict{Tuple{Int16,Int16,Int16},Tuple{Int64,Float64}} with 8205687 entries |


http://juliagraphics.github.io/ColorSchemes.jl/stable/

In [85]:
import Pkg; Pkg.add("ColorSchemes")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed DiffEqBase ─── v6.3.6
 Installed ColorSchemes ─ v3.4.0
  Updating `~/.julia/environments/v1.0/Project.toml`
  [35d6a980] + ColorSchemes v3.4.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [35d6a980] + ColorSchemes v3.4.0
  [2b5f629d] ↑ DiffEqBase v6.3.5 ⇒ v6.3.6


In [86]:
using ColorSchemes

┌ Info: Precompiling ColorSchemes [35d6a980-a343-548e-a6ea-1d62b119f2f4]
└ @ Base loading.jl:1192


In [87]:
ColorSchemes.plasma

In [89]:
# Save to text file
outfile = "data/colored_lorenz_2000px.xyz"
open(outfile, "w") do file
    for (ijk, counts) in cube
        if counts[1] >= 20
            alpha = counts[2]/counts[1]
            rgb = get(ColorSchemes.plasma, alpha)
            println(file, join([ijk..., rgb.r, rgb.g, rgb.b], ' '))
        end
    end
end

Next job is for PotreeConvertor:
- [Install PotreeConvertor](https://github.com/potree/PotreeConverter)

        cmake -DCMAKE_BUILD_TYPE=Release -DLASZIP_INCLUDE_DIRS=~/Projets/strange_attractor/lastools/LASzip/dll -DLASZIP_LIBRARY=~/Projets/strange_attractor/lastools/LASzip/build/src/liblaszip.so ..
        # copy ./PotreeConverter/resources/page_template to your binary working directory.
        
        
- Path to use     PotreeConverter:

        export PATH="../PotreeConvertor/master/build/PotreeConverter/:$PATH"

- Command to convert:

        PotreeConverter -p --page-template ./page_template/ -i lorenz_v02.xyz
    
        PotreeConverter -f xyzrgb -color-range 1 -i colored_lorenz_v01.xyz
        
and for [Potree](http://potree.org/):

--> [a white Lorenz Attractor](https://xdze2.github.io/chaotic-clouds/white_lorenz.html)


![screenshot](./images/thewhitebutterfly.png)

## Draft
using events

In [ ]:
p12 = p2 .- p1
p12_norm = sqrt(sum(p12.^2))
p12 = p12./p12_norm;

function conditions(out, u, t, integrator)
    a1 = u .- p1
    a2 = u .- p2
    out[1] = -sum(-a1.*p12)
    out[2] = -sum(a2.*p12)
end

saved_event_idx = [0, ]
function affect!(integrator, idx)
    saved_event_idx[1] = idx
    terminate!(integrator)
end
#cb = ContinuousCallback(condition, affect!, rootfind=false);
cb = VectorContinuousCallback(conditions, affect!, nothing,
                              2, rootfind=false, save_positions=(true, false))

In [ ]:
u0 = burn()

tspan = (0.0, 200.0)
prob = ODEProblem(lorenz!, u0, tspan)

tol = 1e-8
sol_stop = solve(prob, OwrenZen5(), callback=cb,
                 reltol=tol, abstol=tol, saveat=dt);

saved_event_idx

In [ ]:
sol_stop

In [ ]:
to_ijk.(sol_stop.u)

In [ ]:
plot(sol[2, :], sol[3, :], xlabel="y", ylabel="z")
scatter!([p1[2], ], [p1[3], ], label="p1")
scatter!([p2[2], ], [p2[3], ], label="p2")
plot!(sol_stop[2, :], sol_stop[3, :], color=:red)

In [ ]:
function to_plan()
    
    u0 = burn()
    tspan = (0.0, 160.0)
    prob = ODEProblem(lorenz!, u0, tspan)

    tol = 1e-7
    sol_stop = solve(prob, Tsit5(), callback=cb,
        reltol=tol, abstol=tol);
    return project(sol_stop[end])[1:2]
end

In [ ]:
poincarre = hcat([to_plan() for i in 1:5000]...);

In [ ]:
scatter(poincarre[1, :], poincarre[2, :], markersize=2, markerstrokewidth = 0,
            markeralpha = 0.6, xlabel="y", ylabel="x")
xaxis!(lims=(0, 5))

In [ ]:
u0 = burn()

tspan = (0.0, 1000.0)
prob = ODEProblem(lorenz!, u0, tspan)

tol = 1e-7
sol = solve(prob, Tsit5(), reltol=tol, abstol=tol);

In [ ]:
outfile = "data/lorenz_v01.xyz"
open(outfile, "w") do file
  for s in sol.u
    println(file, "$(s[1]) $(s[2]) $(s[3])")
  end
end

In [ ]:
CSV.write("hello.txt", rand(3, 3))

In [ ]:
?yaxis!#(lims=(1, 6))

In [ ]:
@gif for theta in range(0, stop = 360, length = 150)
    plot3d(sol[1, :], sol[2, :], sol[3, :],
           camera=(theta, 10))
end

In [ ]:
plot(sol[1, :], sol[3, :])

In [ ]:
u0 = [1.0; 0.0; 0.0]
tspan = (0.0, 40.0)
prob = ODEProblem(lorenz!, u0, tspan)
sol1 = solve(prob);
sol2 = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);

In [ ]:
sol_OZ4 = solve(prob, OwrenZen4());
plot(sol_OZ4, linewidth=1)

In [ ]:
sol_RK4 = solve(prob, RK4(), reltol=1e-8, abstol=1e-8);
plot(sol_RK4, linewidth=1)

In [ ]:
plot(sol_RK4[2, :], sol_RK4[3, :])

In [ ]:
plot(sol_RK4.u[], linewidth=1)

In [ ]:
sol_RK4 = solve(prob, OwrenZen3(), reltol=1e-8, abstol=1e-8);
plot(sol_RK4, linewidth=1)

In [ ]:
sol_RK4 = solve(prob, Feagin14(), reltol=1e-8, abstol=1e-8);
plot(sol_RK4, linewidth=1)